# Notebook med API'er som benyttes i beregningene
- Alle spørringer lagres i egen datamappe
- I dette arket hentes kun kapittel. 
- Regionslister hentes i bevilgningsflyten

## Laster inn nødvendige pakker

In [ ]:
import pandas as pd
import requests
import io

## Angi årgang som skal hentes
- Årgangen som angis vil gjelde for hele arket
- Skal det hentes flere årganger må arket kjøres flere ganger

Mappen der filene skal lagres må allerede eksistere

In [ ]:
%run ./../aargang.ipynb

### Kapittel

In [ ]:
#API mot KOSTRA regnskapsarter. 
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/270/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
kap = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
kap = kap[['code', 'validFromInRequestedRange']]
kap.rename(columns={'code':'kapittel', 'validFromInRequestedRange':'periode'}, inplace = True)
kap['periode']=pd.DatetimeIndex(kap['periode']).year
kap = kap[~kap.kapittel.str.contains("K")]
kap.drop_duplicates().astype(str).info()
kap.head(2)

In [ ]:
kap.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kapittel.parquet")